In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from time import sleep
from shutil import rmtree
from pathlib import Path
from os.path import join
import pandas as pd
import numpy as np
import os

from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from sos_trades_core.execution_engine.sos_simple_multi_scenario import SoSSimpleMultiScenario
from sos_trades_core.execution_engine.sos_very_simple_multi_scenario import SoSVerySimpleMultiScenario
from sos_trades_core.execution_engine.scatter_data import SoSScatterData
from sos_trades_core.execution_engine.sos_discipline_scatter import SoSDisciplineScatter
from tempfile import gettempdir
from sos_trades_core.tools.rw.load_dump_dm_data import DirectLoadDump
from sos_trades_core.study_manager.base_study_manager import BaseStudyManager
from sos_trades_core.execution_engine.sos_discipline import SoSDiscipline
from sos_trades_core.execution_engine.sos_coupling import SoSCoupling

# TestScatter

SoSDiscipline test class

# setUp

In [4]:
'''
Initialize third data needed for testing
'''
dirs_to_del = []
namespace = 'MyCase'
study_name = f'{namespace}'
repo = 'sos_trades_core.sos_processes.test'
base_path = 'sos_trades_core.sos_wrapping.test_discs'

root_dir = gettempdir()

In [5]:
exec_eng = ExecutionEngine(namespace)
factory = exec_eng.factory

# tearDown

In [6]:
for dir_to_del in dirs_to_del:
     sleep(0.5)
     if Path(dir_to_del).is_dir():
          rmtree(dir_to_del)
sleep(0.5)

# 01_multi_scenario_of_scatter

In [7]:
exec_eng = ExecutionEngine(namespace)
factory = exec_eng.factory

In [8]:
# scatter build map
ac_map = {'input_name': 'name_list',
            'input_type': 'string_list',
            'input_ns': 'ns_scatter_scenario',
            'output_name': 'ac_name',
            'scatter_ns': 'ns_ac',
            'gather_ns': 'ns_scenario',
            'ns_to_update': ['ns_data_ac']}

exec_eng.smaps_manager.add_build_map('name_list', ac_map)
import pandas as pd
pd.DataFrame.from_dict(ac_map ,orient='index')

,0
input_name,name_list
input_type,string_list
input_ns,ns_scatter_scenario
output_name,ac_name
scatter_ns,ns_ac
gather_ns,ns_scenario
ns_to_update,[ns_data_ac]


In [9]:
# scenario build map
scenario_map = {'input_name': 'scenario_list',
                    'input_type': 'string_list',
                    'input_ns': 'ns_scatter_scenario',
                    'output_name': 'scenario_name',
                    'scatter_ns': 'ns_scenario',
                    'gather_ns': 'ns_scatter_scenario',
                    'ns_to_update': ['ns_disc3', 'ns_barrierr', 'ns_out_disc3']}

exec_eng.smaps_manager.add_build_map(
     'scenario_list', scenario_map)
import pandas as pd
pd.DataFrame.from_dict(scenario_map ,orient='index')

,0
input_name,scenario_list
input_type,string_list
input_ns,ns_scatter_scenario
output_name,scenario_name
scatter_ns,ns_scenario
gather_ns,ns_scatter_scenario
ns_to_update,"[ns_disc3, ns_barrierr, ns_out_disc3]"


In [10]:
# shared namespace
exec_eng.ns_manager.add_ns('ns_barrierr', 'MyCase')
exec_eng.ns_manager.add_ns(
     'ns_scatter_scenario', 'MyCase.multi_scenarios')
exec_eng.ns_manager.add_ns(
     'ns_disc3', 'MyCase.multi_scenarios.Disc3')
exec_eng.ns_manager.add_ns(
     'ns_out_disc3', 'MyCase.multi_scenarios')
exec_eng.ns_manager.add_ns(
     'ns_data_ac', 'MyCase')

In [11]:
# instantiate factory # get instantiator from Discipline class

builder_list = factory.get_builder_from_process(repo=repo,
                                                                  mod_id='test_disc1_scenario')

scatter_list = exec_eng.factory.create_multi_scatter_builder_from_list(
     'name_list', builder_list=builder_list, autogather=True)

mod_list = f'{base_path}.disc3_scenario.Disc3'
disc3_builder = exec_eng.factory.get_builder_from_module(
     'Disc3', mod_list)
scatter_list.append(disc3_builder)

multi_scenarios = exec_eng.factory.create_very_simple_multi_scenario_builder(
     'multi_scenarios', 'scenario_list', scatter_list, autogather=True, gather_node='Post-processing')

exec_eng.factory.set_builders_to_coupling_builder(
     multi_scenarios)
exec_eng.configure()
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase is set to MDAJacobi
INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
	|_ Post-processing
		|_ Disc1
		|_ Disc3


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3'

In [12]:
scatter_list[1].__dict__


{'_SoSBuilder__disc_name': 'Disc1',
 'disc': None,
 '_SoSBuilder__ee': <sos_trades_core.execution_engine.execution_engine.ExecutionEngine at 0x1ac11866460>,
 '_SoSBuilder__args': {'sos_name': 'Disc1',
  'ee': <sos_trades_core.execution_engine.execution_engine.ExecutionEngine at 0x1ac11866460>,
  'map_name': 'name_list',
  'cls_builder': <sos_trades_core.execution_engine.sos_builder.SoSBuilder at 0x1ac15aac1c0>},
 'cls': sos_trades_core.execution_engine.sos_discipline_gather.SoSDisciplineGather,
 'discipline_dict': {},
 '_is_executable': True}

In [13]:
builder_list[0]

In [14]:
multi_scenarios[1].__dict__

{'_SoSBuilder__disc_name': 'Post-processing.Disc1',
 'disc': Post-processing.Disc1
    Inputs: MyCase.multi_scenarios.scenario_list
    Outputs: ,
 '_SoSBuilder__ee': <sos_trades_core.execution_engine.execution_engine.ExecutionEngine at 0x1ac11866460>,
 '_SoSBuilder__args': {'sos_name': 'Post-processing.Disc1',
  'ee': <sos_trades_core.execution_engine.execution_engine.ExecutionEngine at 0x1ac11866460>,
  'map_name': 'scenario_list',
  'cls_builder': <sos_trades_core.execution_engine.sos_builder.SoSBuilder at 0x1ac15aace20>},
 'cls': sos_trades_core.execution_engine.sos_discipline_gather.SoSDisciplineGather,
 'discipline_dict': {'MyCase.Post-processing.Disc1': Post-processing.Disc1
     Inputs: MyCase.multi_scenarios.scenario_list
     Outputs: },
 '_is_executable': True}

In [15]:
from copy import deepcopy
DESC_IN = deepcopy(builder_list[0].cls.DESC_IN)
DESC_OUT = deepcopy(builder_list[0].cls.DESC_OUT)

In [16]:
import pandas as pd
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [17]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_data_ac
a,float,Shared,ns_data_ac
b,float,NaN,NaN


In [18]:
DESC_OUT_df

,type,visibility,namespace
indicator,float,NaN,NaN
y,float,Shared,ns_ac


In [19]:
from copy import deepcopy
DESC_IN = deepcopy(disc3_builder.cls.DESC_IN)
DESC_OUT = deepcopy(disc3_builder.cls.DESC_OUT)
import pandas as pd
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [20]:
DESC_IN_df

,type,visibility,namespace
z,float,Shared,ns_disc3
constant,float,NaN,NaN
power,int,NaN,NaN


In [21]:
DESC_OUT_df

,type,visibility,namespace
o,float,Shared,ns_out_disc3


In [22]:
dict_values = {}
dict_values[f'{study_name}.multi_scenarios.scenario_list'] = [
     'scenario_1', 'scenario_2']
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_1', 'scenario_2']}

In [23]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.

INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of t

INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of t

INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of t

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.

ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_2.Disc3.z value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_2.Disc3.constant value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_2.Disc3.power value is not set!
INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
		|_ scenario_1
			|_ Disc1
			|_ Disc3
		|_ scenario_2
			|_ Disc1
			|_ Disc3
	|_ Post-processing
		|_ Disc1
		|_ Disc3


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3'

In [24]:
scenario_list = ['scenario_1', 'scenario_2']
for scenario in scenario_list:
     x1 = 2.
     x2 = 4.
     a1 = 3
     b1 = 4
     a2 = 6
     b2 = 2

     dict_values[study_name + '.name_1.a'] = a1
     dict_values[study_name + '.name_2.a'] = a2
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc1.name_1.b'] = b1
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc1.name_2.b'] = b2
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc3.constant'] = 3
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc3.power'] = 2

dict_values[study_name +
               '.multi_scenarios.name_list'] = ['name_1', 'name_2']
dict_values[study_name +
               '.multi_scenarios.scenario_1.Disc3.z'] = 1.2
dict_values[study_name +
               '.multi_scenarios.scenario_2.Disc3.z'] = 1.5
dict_values[study_name + '.name_1.x'] = x1
dict_values[study_name + '.name_2.x'] = x2
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_1', 'scenario_2'],
 'MyCase.name_1.a': 3,
 'MyCase.name_2.a': 6,
 'MyCase.multi_scenarios.scenario_1.Disc1.name_1.b': 4,
 'MyCase.multi_scenarios.scenario_1.Disc1.name_2.b': 2,
 'MyCase.multi_scenarios.scenario_1.Disc3.constant': 3,
 'MyCase.multi_scenarios.scenario_1.Disc3.power': 2,
 'MyCase.multi_scenarios.scenario_2.Disc1.name_1.b': 4,
 'MyCase.multi_scenarios.scenario_2.Disc1.name_2.b': 2,
 'MyCase.multi_scenarios.scenario_2.Disc3.constant': 3,
 'MyCase.multi_scenarios.scenario_2.Disc3.power': 2,
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2'],
 'MyCase.multi_scenarios.scenario_1.Disc3.z': 1.2,
 'MyCase.multi_scenarios.scenario_2.Disc3.z': 1.5,
 'MyCase.name_1.x': 2.0,
 'MyCase.name_2.x': 4.0}

In [25]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.

'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [26]:
exec_eng.execute()

INFO:SoS.EE:PROCESS EXECUTION MyCase STARTS...
INFO:SoS.EE:PROCESS EXECUTION MyCase ENDS.


{'MyCase.name_1.x': array([2.]),
 'MyCase.name_1.a': array([3]),
 'MyCase.multi_scenarios.scenario_1.Disc1.name_1.b': array([4]),
 'MyCase.name_2.x': array([4.]),
 'MyCase.name_2.a': array([6]),
 'MyCase.multi_scenarios.scenario_1.Disc1.name_2.b': array([2]),
 'MyCase.multi_scenarios.name_list': [1, 1],
 'MyCase.multi_scenarios.scenario_1.Disc3.z': array([1.2]),
 'MyCase.multi_scenarios.scenario_1.Disc3.constant': array([3]),
 'MyCase.multi_scenarios.scenario_1.Disc3.power': array([2]),
 'MyCase.multi_scenarios.scenario_2.Disc1.name_1.b': array([4]),
 'MyCase.multi_scenarios.scenario_2.Disc1.name_2.b': array([2]),
 'MyCase.multi_scenarios.scenario_2.Disc3.z': array([1.5]),
 'MyCase.multi_scenarios.scenario_2.Disc3.constant': array([3]),
 'MyCase.multi_scenarios.scenario_2.Disc3.power': array([2]),
 'MyCase.multi_scenarios.scenario_list': [1, 1],
 'MyCase.multi_scenarios.scenario_1.Disc1.name_1.indicator': array([12]),
 'MyCase.multi_scenarios.scenario_1.name_1.y': array([10.]),
 'MyCas

In [27]:
print(exec_eng.dm.get_value(
     'MyCase.multi_scenarios.scenario_list'), ['scenario_1', 'scenario_2'])

['scenario_1', 'scenario_2'] ['scenario_1', 'scenario_2']


In [28]:
y1 = a1 * x1 + b1
y2 = a2 * x2 + b2

In [29]:
print(exec_eng.dm.get_value(
     'MyCase.multi_scenarios.scenario_1.name_1.y'), y1)
print(exec_eng.dm.get_value(
     'MyCase.multi_scenarios.scenario_1.name_2.y'), y2)
print(exec_eng.dm.get_value(
     'MyCase.multi_scenarios.scenario_2.name_1.y'), y1)
print(exec_eng.dm.get_value(
     'MyCase.multi_scenarios.scenario_2.name_2.y'), y2)

10.0 10.0
26.0 26.0
10.0 10.0
26.0 26.0


In [30]:
gather_disc1 = exec_eng.dm.get_disciplines_with_name(
     'MyCase.Post-processing.Disc1')[0]

In [31]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
		|_ scenario_1
			|_ Disc1
				|_ name_1
				|_ name_2
			|_ Disc3
		|_ scenario_2
			|_ Disc1
				|_ name_1
				|_ name_2
			|_ Disc3
	|_ Post-processing
		|_ Disc1
		|_ Disc3
	|_ name_1
	|_ name_2


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [32]:
print([key for key in list(gather_disc1._data_in.keys()) if key not in gather_disc1.NUM_DESC_IN], [
     'scenario_list', 'scenario_2.y_dict', 'scenario_1.y_dict'])
print(list(gather_disc1._data_out.keys()), ['y_dict'])
print(gather_disc1._data_out['y_dict']['value'], {
                          'scenario_1.name_1': y1, 'scenario_1.name_2': y2, 'scenario_2.name_1': y1, 'scenario_2.name_2': y2})

['scenario_list', 'scenario_1.y_dict', 'scenario_2.y_dict'] ['scenario_list', 'scenario_2.y_dict', 'scenario_1.y_dict']
['y_dict'] ['y_dict']
{'scenario_1.name_1': 10.0, 'scenario_1.name_2': 26.0, 'scenario_2.name_1': 10.0, 'scenario_2.name_2': 26.0} {'scenario_1.name_1': 10.0, 'scenario_1.name_2': 26.0, 'scenario_2.name_1': 10.0, 'scenario_2.name_2': 26.0}


In [33]:
gather_disc3 = exec_eng.dm.get_disciplines_with_name(
     'MyCase.Post-processing.Disc3')[0]

In [34]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
		|_ scenario_1
			|_ Disc1
				|_ name_1
				|_ name_2
			|_ Disc3
		|_ scenario_2
			|_ Disc1
				|_ name_1
				|_ name_2
			|_ Disc3
	|_ Post-processing
		|_ Disc1
		|_ Disc3
	|_ name_1
	|_ name_2


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [35]:
print([key for key in list(gather_disc3._data_in.keys())if key not in SoSDiscipline.NUM_DESC_IN], [
     'scenario_list', 'scenario_1.o', 'scenario_2.o'])
print(list(gather_disc3._data_out.keys()), ['o_dict'])

['scenario_list', 'scenario_1.o', 'scenario_2.o'] ['scenario_list', 'scenario_1.o', 'scenario_2.o']
['o_dict'] ['o_dict']
